<a href="https://colab.research.google.com/github/vitamingyu/NLP-LLM/blob/main/tf_43attension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# 어텐션 : 디코더에서 출력단어를 예측하는데 매 시점마다 인코더에서 전체 입력문장을 다시 참조하는 방식
# 해당시점에서 예측해야 할 단어와 연관이 있는 입력 단어에 좀 더 집중해서 작업한다.
# seq2seq 알고리즘에 문제점 중 일부를 개선
import tensorflow as tf
from keras.layers import Input, LSTM, Dense, Concatenate, Attention
from keras.models import Model

In [13]:
# 가상의 파라미터에 대한 초기값
input_length = 10
output_length = 10
vocab_size = 100
embedding_dim = 64
lstm_units = 128

# encoder정의
encoder_inputs = Input(shape=(input_length, embedding_dim))
encoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
encoder_outputs,_,_ = encoder_lstm(encoder_inputs)

# decoder정의
decoder_inputs = Input(shape=(output_length, embedding_dim))
decoder_lstm = LSTM(lstm_units, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs)

# Attension 레이어
attention_layer = Attention()
attention_output = attention_layer([decoder_outputs, encoder_outputs])

# Attention 레이어는 디코더의 출력과 인코더의 출력 사이에 관계를 계산하여 중요 정보에 집중할 수 있도록 도움을 준다
concat_layer = Concatenate(axis= -1)
decoder_concat_input = concat_layer([decoder_outputs, attention_output])

# 출력 레이어 : 최종적으로 Dense를 통해 예측을 수행한다
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_12 (InputLayer)       [(None, 10, 64)]             0         []                            
                                                                                                  
 input_11 (InputLayer)       [(None, 10, 64)]             0         []                            
                                                                                                  
 lstm_11 (LSTM)              (None, 10, 128)              98816     ['input_12[0][0]']            
                                                                                                  
 lstm_10 (LSTM)              [(None, 10, 128),            98816     ['input_11[0][0]']            
                              (None, 128),                                                    